# Teoria

**Probabilità condizionata**

misura della probabilità che un evento accada dato un altro evento già accaduto $P(B|A)$ probabilità che accada B dato A

**Teorema di Bayes**

$P(A|B) = \frac{P(A \bigcap B)}{P(B)} = \frac{P(A)*P(B|A)}{P(B)}$ 

dove $P(A \cap B)$ è la probabilità che avvengano entrambi

**modelli e algoritmi naive bayes**

due importanti assunzioni su dati:
- feature tra loro indipendenti
- feature hanno tutte lo stesso peso

$P(y|x_1,...,x_n)=\frac{P(y)P(x_1,...,x_n)}{P(x_1,...,x_n)} = \frac{P(y)P(x1|y)...P(x_n|y)}{P(x1)...P(x_n)} = \frac{P(y)\prod_{i=1}^n P(x_i|y)}{P(x1)...P(x_n)}$

il denominatore è costante per qualunque valore di y, lo posso rimuovere usando il simbolo $\propto$

$P(y|x_1,...,x_n) \propto P(y)\prod_{i=1}^n P(x_i|y)$

Hai osservato in passato:

La parola “gratis” compare spesso in email spam.

La parola “ciao” compare più spesso nelle email normali.

Ora arriva un’email con scritto:

"Ciao! Hai vinto un premio gratis!"

Il modello guarda le tre parole:
ciao, vinto, gratis.

E pensa:

“Ciao” → di solito è in email non spam.

“Gratis” → quasi sempre in email spam.

“Vinto” → spesso in email spam.

Allora conclude:

"Mmh... più probabilmente è spam, quindi la classifico così."

🧠 Perché si chiama Naive (ingenuo)?
Perché fa finta che ogni parola sia indipendente dalle altre.
Cioè pensa che il fatto che ci sia “gratis” non influenzi il fatto che ci sia anche “vinto”.

Nella realtà questo non è vero, ma sorprendentemente... funziona lo stesso molto bene!

**regola di classificazione** <br>
$ y = \argmax_y P(y) \prod_{i=1}^n P(x_i|y)$

- utilizzabile solo per dati discreti
- tende all'overfitting (poiche si basa troppo su dati di training)

si fanno assunzioni su $P(x|y)$, i diversi algoritmi NB differiscono in questo

**Multinomial NB**

$P(x_i|y) = \frac{N_{yi}+\alpha}{N_y+\alpha n}$

valido per dati multinomiali
